In [43]:
import torch
import torchvision
from torchvision.models.detection.transform import GeneralizedRCNNTransform
from torchvision.models.detection.backbone_utils import mobilenet_backbone

from torchvision.models.detection.anchor_utils import AnchorGenerator
nn = torch.nn
import os
import numpy as np
from src.utils.audio_utils import compute_mel_spectrogram, compute_spectrogram, load_audio_file
from src.utils import path_utils


In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [78]:
class OneShotDataset(torch.utils.data.Dataset):
    def __init__(self, detection_dir,transform_audio, device):
        self.detection_dir = detection_dir
        self.device = device
        self.samplepath = os.path.join(self.detection_dir, 'samples')
        self.targetpath = os.path.join(self.detection_dir, 'target')
        self.indexes = list(map(lambda x: x.split('.')[0],os.listdir(self.samplepath)))
        self.transform_audio = transform_audio
        self.transform_image = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
        self.spec_hight=120

    def __len__(self):
        return len(os.listdir((self.samplepath)))
        
    def __getitem__(self,idx):
        index=self.indexes[idx]
        data,fs = load_audio_file(os.path.join(self.samplepath,index +'.wav'))
        target = np.load(os.path.join(self.targetpath,index +'.npy'))
        bbox=[[cls,xs,0,xe,self.spec_hight] for [cls,xs,xe]in target]
        print(bbox)
        #  bbox=[[target[0],target[1],0,target[2],self.spec_hight] for  target in targets]

        x = self.transform_audio(data)
        x = self.transform_image(x)
        return x, bbox

In [79]:
def transform_audio(data):
    _, _, specto = compute_spectrogram(data, 24000, nperseg=256, noverlap=256/2, scale="dB")
    # freq clip
    specto = specto[:120, :]
    return specto
    
detection_dataloader = torch.utils.data.DataLoader(OneShotDataset(detection_dir=path_utils.get_detection_data_path(),
                                                    transform_audio=transform_audio, device=device),batch_size=1)
                                                    

In [81]:
for (x,y)in detection_dataloader:
    pass

[]
[]
[]
[]
[]
[]
[[0, 1638, 0, 44090, 120]]
[[0, 4553, 0, 43908, 120]]
[]
[[1, 17817, 0, 36119, 120], [1, 19995, 0, 43395, 120]]
[]
[[1, 7540, 0, 37059, 120], [1, 9072, 0, 38563, 120]]
[]
[]
[[0, 1985, 0, 47987, 120]]
[]
[[0, 1585, 0, 43084, 120], [0, 3818, 0, 47034, 120]]
[]
[]
[[0, 2001, 0, 46826, 120]]
[]
[[1, 13302, 0, 44797, 120]]
[[0, 1717, 0, 46597, 120]]
[]
[]
[[0, 352, 0, 47957, 120]]
[]
[]
[[0, 6425, 0, 47708, 120]]
[[0, 365, 0, 47287, 120]]
[]
[[1, 23730, 0, 46873, 120]]
[]
[[1, 7341, 0, 42266, 120]]
[]
[]
[[1, 9986, 0, 29587, 120], [1, 7408, 0, 24942, 120]]
[]
[[0, 3559, 0, 44701, 120]]
[[0, 8433, 0, 46906, 120]]
[[1, 17810, 0, 40052, 120], [0, 487, 0, 46093, 120]]
[]
[]
[]
[[0, 2169, 0, 47021, 120]]
[[1, 6376, 0, 32335, 120], [0, 7628, 0, 47490, 120]]
[]
[]
[]
[]
[]
[]
[[1, 8055, 0, 41819, 120]]
[]
[[1, 8014, 0, 38541, 120]]
[[1, 13313, 0, 45948, 120]]
[[1, 10052, 0, 42880, 120]]
[[1, 8887, 0, 35579, 120], [0, 5421, 0, 45365, 120]]
[[0, 2744, 0, 45143, 120]]
[]
[]
[[0, 16

In [46]:

def rcnn_pretrained_backbone(num_classes : int, anchor_sizes : tuple, aspect_ratios : tuple, parameters : dict = {}):
    """
    Return a faster rcnn with a mobilenetv3 backbone.

    Args:
        num_classes (int): Number of classes expected to return (background should be taken into account)
        parameters (dict, optional): Dictionnary for the different following parameters. Defaults to {}.

    Returns:
        (torchvision.models.detection.faster_rcnn.FasterRCNN): Model implementation of pytorch

    >>> training
    # RPN
    rpn_pre_nms_top_n_train (int): number of proposals to keep before applying NMS during training
    rpn_post_nms_top_n_train (int): number of proposals to keep after applying NMS during training
    rpn_fg_iou_thresh (float): minimum IoU between the anchor and the GT box so that they can be
        considered as positive during training of the RPN.
    rpn_bg_iou_thresh (float): maximum IoU between the anchor and the GT box so that they can be
        considered as negative during training of the RPN.
    rpn_batch_size_per_image (int): number of anchors that are sampled during training of the RPN
        for computing the loss
    rpn_positive_fraction (float): proportion of positive anchors in a mini-batch during training
        of the RPN

    # Head
    box_fg_iou_thresh (float): minimum IoU between the proposals and the GT box so that they can be
        considered as positive during training of the classification head
    box_bg_iou_thresh (float): maximum IoU between the proposals and the GT box so that they can be
        considered as negative during training of the classification head
    box_batch_size_per_image (int): number of proposals that are sampled during training of the
        classification head
    box_positive_fraction (float): proportion of positive proposals in a mini-batch during training
        of the classification head



    >>> inference
    # RPN
    rpn_pre_nms_top_n_test (int): number of proposals to keep before applying NMS during testing
    rpn_post_nms_top_n_test (int): number of proposals to keep after applying NMS during testing
    rpn_score_thresh (float): during inference, only return proposals with a classification score
        greater than rpn_score_thresh
    rpn_nms_thresh (float): NMS threshold used for postprocessing the RPN proposals

    # Head
    box_score_thresh (float): during inference, only return proposals with a classification score
        greater than box_score_thresh

    box_nms_thresh (float): NMS threshold for the prediction head. Used during inference
    box_detections_per_img (int): maximum number of detections per image, for all classes.

    """

    pretrained_backbone=True
    trainable_backbone_layers=6 # All backbone is trainable

    backbone = mobilenet_backbone("mobilenet_v3_large", pretrained_backbone, True, trainable_layers=trainable_backbone_layers)

    model = torchvision.models.detection.faster_rcnn.FasterRCNN(backbone,
                                                                num_classes,
                                                                rpn_anchor_generator=AnchorGenerator(anchor_sizes, aspect_ratios),
                                                                **parameters)

    # Custum transform ie no transform (only postprocessing)
    model.transform = GeneralizedRCNNTransform(min_size=489, max_size=2000, image_mean=[0, 0, 0], image_std=[1, 1, 1])
    return model

In [47]:
def rcnn_pretrained_backbone_train(num_classes : int, anchor_sizes : tuple, aspect_ratios : tuple, parameters : dict = {}):
    """
    Return a jit compiled faster rcnn with a mobilenetv3 backbone.
    """
    model = rcnn_pretrained_backbone(num_classes, anchor_sizes, aspect_ratios, parameters)
    model._has_warned = True # Remove warning about "RCNN always returns a (Losses, Detections) tuple in scripting"
    return torch.jit.script(model)

In [48]:
def loss_faster_rcnn(dict_losses : dict, training_rpn : bool, training_head : bool):
    """ Reduce the dictionnary of losses

    Args:
        dict_losses (dict): Dictionnary of losses
        training_rpn (bool): Bool to train the rpn
        training_head (bool): Bool to train the head

    Returns:
        (torch.Tensor): Loss
    """
    # Dict("loss_classifier", "loss_box_reg", "loss_objectness", "loss_rpn_box_reg")
    loss = torch.zeros((), dtype=torch.float32, device=device)
    if training_rpn:
        loss += dict_losses["loss_objectness"]
        loss += dict_losses["loss_rpn_box_reg"]
    if training_head:
        loss += dict_losses["loss_classifier"]
        loss += dict_losses["loss_box_reg"]

In [49]:
def average_losses(dataloader, mean_loss):
    mean_loss["loss_objectness"] /= len(dataloader)
    mean_loss["loss_rpn_box_reg"] /= len(dataloader)
    mean_loss["loss_classifier"] /= len(dataloader)
    mean_loss["loss_box_reg"] /= len(dataloader)
    return mean_loss

def accumulate_losses(mean_loss, losses):
    mean_loss["loss_objectness"] += losses["loss_objectness"].item()
    mean_loss["loss_rpn_box_reg"] += losses["loss_rpn_box_reg"].item()
    mean_loss["loss_classifier"] += losses["loss_classifier"].item()
    mean_loss["loss_box_reg"] += losses["loss_box_reg"].item()

In [62]:
def transform_audio(data):
    _, _, specto = compute_spectrogram(data, 24000, nperseg=256, noverlap=256/2, scale="dB")
    # freq clip
    specto = specto[:120, :]
    return specto
    
detection_dataloader = torch.utils.data.DataLoader(OneShotDataset(detection_dir=path_utils.get_detection_data_path(),
                                                    transform_audio=transform_audio, device=device),batch_size=1)
                                                    

In [63]:
model = rcnn_pretrained_backbone_train(num_classes=4,anchor_sizes=[32,64,128],aspect_ratios=[0.5,1,2])
model.to(device)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
sgd_optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)

# and a learning rate scheduler which decreases the learning rate by
# 10x every 3 epochs
lr_scheduler = torch.optim.lr_scheduler.StepLR(sgd_optimizer,
                                               step_size=3,
                                               gamma=0.1)
                                        
loss_fn_frrcnn=lambda x:loss_faster_rcnn(x,True,True)

In [66]:
def train_loop(dataloader, model, loss_fn, optimizer, scheduler, macro_batch=1):
    # Initialize training
    model.train()
    optimizer.zero_grad()

    mean_loss = {"loss_objectness": 0., "loss_rpn_box_reg": 0., "loss_classifier": 0., "loss_box_reg": 0.}

    # Iterate over the dataset
    for batch, (X, targets) in enumerate(dataloader):
        # Work with the GPU if available
        X = list(x.to(device) for x in X)
        #targets = list({k: v.to(device) for k, v in t.item()} for t in targets)

        # Compute prediction error
        losses, detections = model(X, targets)
        accumulate_losses(mean_loss, losses)
        loss = loss_fn(losses)

        # Backpropagation
        loss.backward()
        if (batch+1) % macro_batch == 0 or batch == len(dataloader) - 1:
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()

        # Print metrics
        if batch % 30 == 0 or batch == len(dataloader) - 1:
            loss_value, current = loss.item(), (batch+1) * len(X)
        print(loss_value)
    return average_losses(dataloader, mean_loss)

In [67]:
train_loop(dataloader=detection_dataloader, model=model, loss_fn=loss_fn_frrcnn, optimizer=sgd_optimizer, scheduler=lr_scheduler, macro_batch=1)


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
  File "/Users/paulbp_dty/Desktop/P3_BBF/audio_env/lib/python3.10/site-packages/torchvision/models/detection/generalized_rcnn.py", line 65, in forward
            assert targets is not None
            for target in targets:
                boxes = target["boxes"]
                        ~~~~~~~~~~~~~~ <--- HERE
                if isinstance(boxes, torch.Tensor):
                    if len(boxes.shape) != 2 or boxes.shape[-1] != 4:
RuntimeError: KeyError: boxes
